In [1]:
import pandas as pd
import os
pd.set_option('display.max_columns', None)

# Definimos las rutas
ruta_historial = '../data/raw/New_HistConsultas.csv'
ruta_usuarios = '../data/raw/New_Usuarios.csv'

In [2]:
# Carga de archivos.
try:
    # Encoding='latin-1' para que lea los caracteres en español
    # sep=';' 
    df_consultas_raw = pd.read_csv(ruta_historial, encoding='latin-1', sep=';') 
    df_usuarios_raw = pd.read_csv(ruta_usuarios, encoding='latin-1', sep=';')
    print("Las bases cargaron bien.")
except Exception as e:
    print("Hubo un error cargando los archivos")
    print(e)

# Revision rapida para ver que columnas trajo y si cruzan bien
if 'df_consultas_raw' in locals() and 'df_usuarios_raw' in locals():
    print("Dimensiones Historial:", df_consultas_raw.shape)
    print("Dimensiones Usuarios:", df_usuarios_raw.shape)

    print("Columnas Historial:", df_consultas_raw.columns.tolist())
    print("Columnas Usuarios:", df_usuarios_raw.columns.tolist())

Las bases cargaron bien.
Dimensiones Historial: (1282960, 4)
Dimensiones Usuarios: (337051, 22)
Columnas Historial: ['Unnamed: 0', 'ID_Cuenta', 'Fecha_consulta', 'Tipo_consulta']
Columnas Usuarios: ['ID_Cuenta', 'Tipo_persona', 'Departamento', 'Tiene_plan_avanzado', 'Tipo_Plan', 'Es_moroso', 'Tiene_plus', 'Ha_caido_mora', 'Correo', 'usa_app', 'Forma_pago', 'Monto_adeudado', 'Edad', 'Estrato', 'Motivo_llamada', 'Duracion_llamada', 'Tiempo_en_espera', 'Transferencia_llamada', 'primera_llamada', 'Antiguedad', 'Recomienda_marca', 'y']


In [21]:
# Muestro las primeras filas para validar el contenido
display(df_consultas_raw.head())

,Unnamed: 0,ID_Cuenta,Fecha_consulta,Tipo_consulta
0,0,340,2015-06-22 16:14:35,Actualización datos
1,1,340,2015-06-22 16:37:02,Agendamiento citas
2,2,340,2015-06-22 15:50:56,Agendamiento citas
3,3,4d5c0e16-9bef-8334-687e-55f043e02c62,2015-09-09 14:35:37,Agendamiento citas
4,4,66df1d0e-0d2c-1e44-aa3a-55b2a013640c,2015-07-24 20:36:26,Agendamiento citas


In [20]:
# Muestro las primeras filas para validar el contenido
display(df_usuarios_raw.head())

,ID_Cuenta,Tipo_persona,Departamento,Tiene_plan_avanzado,Tipo_Plan,Es_moroso,Tiene_plus,Ha_caido_mora,Correo,usa_app,Forma_pago,Monto_adeudado,Edad,Estrato,Motivo_llamada,Duracion_llamada,Tiempo_en_espera,Transferencia_llamada,primera_llamada,Antiguedad,Recomienda_marca,y
0,340,soltero,Santafé de Bogotá,si,f,no,no,no,no,si,mensajero,16475.149938,36,NaN,m21,114.522845,15.663706,no,no,mid-age,no,0
1,4d5c0e16-9bef-8334-687e-55f043e02c62,soltero,Santafé de Bogotá,no,f,no,si,si,si,no,mensajero,30167.091798,65,3,m17,124.262540,55.257504,no,no,new-new,si,0
2,66df1d0e-0d2c-1e44-aa3a-55b2a013640c,soltero,Santafé de Bogotá,no,f,no,si,no,no,si,tienda,11842.044402,30,2,m10,178.120444,4.635882,no,no,mid-age,si,0
3,1640de7c-ba08-cdfc-c21b-517fd7c5a259,soltero,Santafé de Bogotá,no,f,si,si,no,no,si,online,39601.288181,32,4,m18,332.970456,30.698536,no,no,Legend,no,0
4,e820f090-f4e7-eb8b-677c-55808c868b6f,unión libre,Santafé de Bogotá,no,b,no,si,si,si,si,online,36126.943210,70,3,m2,70.864127,44.999640,si,si,Legend,no,0


In [5]:
# Vamos a agrupar por ID para ver cuantos usuarios DIFERENTES tienen el mismo ID.
# Si el ID es unico, el conteo deberia ser 1.

conteo_duplicados = df_usuarios_raw['ID_Cuenta'].value_counts()

# Filtramos solo los que aparecen mas de una vez
ids_problematicos = conteo_duplicados[conteo_duplicados > 1]

print(f"Total de IDs que estan repetidos en la base de Usuarios: {len(ids_problematicos)}")
print("Top 10 de IDs mas repetidos (posibles IDs genericos o basura):")
print(ids_problematicos.head(10))

Total de IDs que estan repetidos en la base de Usuarios: 16878
Top 10 de IDs mas repetidos (posibles IDs genericos o basura):
ID_Cuenta
121314        14
1234          13
1022390282     8
1              7
30881871       6
ANONIMO1       5
12345          4
23621188       4
900475036      4
93436751       4
Name: count, dtype: int64


In [6]:
# Tomemos el caso mas critico (el ID '1' o '1234') para ver sus diferencias
id_ejemplo = '1'  # O usa '1234' si prefieres

print(f"Perfiles encontrados para el ID '{id_ejemplo}':")

# Seleccionamos columnas clave para ver que son personas distintas
cols_clave = ['ID_Cuenta', 'Departamento', 'Edad', 'Estado_Civil', 'Tipo_Plan'] 
# Ajusta 'Estado_Civil' si la columna se llama diferente en tu df real

subset_problematico = df_usuarios_raw[df_usuarios_raw['ID_Cuenta'] == id_ejemplo]
display(subset_problematico)

print("Conclusion: El mismo ID tiene edades y ubicaciones diferentes. No se puede cruzar.")

Perfiles encontrados para el ID '1':


,ID_Cuenta,Tipo_persona,Departamento,Tiene_plan_avanzado,Tipo_Plan,Es_moroso,Tiene_plus,Ha_caido_mora,Correo,usa_app,Forma_pago,Monto_adeudado,Edad,Estrato,Motivo_llamada,Duracion_llamada,Tiempo_en_espera,Transferencia_llamada,primera_llamada,Antiguedad,Recomienda_marca,y
7,1,casado,Exterior,si,NaN,si,no,no,si,si,NaN,3620.036707,71,1,m33,252.918581,56.022423,no,si,Young,si,0
8,1,casado,Boyacá,no,d,no,si,si,no,no,online,14410.181689,45,4,m17,168.056028,2.801818,si,no,Legend,si,0
9,1,casado,Cauca,no,b,no,si,si,si,si,online,9877.311198,78,5,m32,53.254917,26.057464,no,si,new-new,si,1
10,1,casado,Santafé de Bogotá,no,f,no,si,no,no,si,online,10653.923985,19,5,m11,194.760263,11.764282,no,no,Young,si,0
11,1,soltero-casado.,Antioquia,no,e,no,si,no,si,si,online,15594.096321,45,4,m32,396.918333,20.904321,no,no,Young,talvez,1
12,1,casado,Valle del Cauca,no,e,no,si,no,no,si,NaN,7759.609790,52,1,m10,117.241591,5.882674,si,no,Legend,no,0
13,1,casado,Cundinamarca,no,e,no,si,no,no,si,NaN,10944.635170,63,3,m9,83.091281,11.414151,si,si,new-new,si,1


Conclusion: El mismo ID tiene edades y ubicaciones diferentes. No se puede cruzar.


In [10]:
# Paso 1: Identificar IDs que son unicos de verdad
ids_unicos_reales = conteo_duplicados[conteo_duplicados == 1].index

# Paso 2: Filtrar la base de usuarios para dejar solo los que tienen IDs confiables
df_usuarios_clean = df_usuarios_raw[df_usuarios_raw['ID_Cuenta'].isin(ids_unicos_reales)].copy()

print(f"Usuarios originales: {df_usuarios_raw.shape[0]}")
print(f"Usuarios con IDs confiables: {df_usuarios_clean.shape[0]}")
print(f"Registros ambiguos eliminados: {df_usuarios_raw.shape[0] - df_usuarios_clean.shape[0]}")

# Ahora si podriamos hacer el merge con df_usuarios_clean

Usuarios originales: 337051
Usuarios con IDs confiables: 302984
Registros ambiguos eliminados: 34067


In [13]:
# 1. Validación de Formato de IDs
# Revisamos si hay caracteres extraños o longitudes inconsistentes en ID_Cliente
# (Asumiendo que ID_Cliente es la llave, si es ID_Cuenta ajustamos la variable)
print("--- Análisis de IDs en Consultas ---")
print(f"Total registros: {len(df_consultas_raw)}")
print(f"IDs únicos de clientes que consultaron: {df_consultas_raw['ID_Cuenta'].nunique()}")

# Chequeo de IDs nulos
ids_nulos = df_consultas_raw['ID_Cuenta'].isnull().sum()
print(f"Registros sin ID de cliente: {ids_nulos}")

# Chequeo de IDs con formatos sospechosos (ej. longitudes diferentes a la moda)
# Esto detecta si hay IDs tipo 'R-234' mezclados con '234'
df_consultas_raw['len_id'] = df_consultas_raw['ID_Cuenta'].astype(str).apply(len)
print("\nDistribución de longitud de los IDs:")
print(df_consultas_raw['len_id'].value_counts())

# Eliminamos la columna auxiliar
df_consultas_raw.drop(columns=['len_id'], inplace=True)

--- Análisis de IDs en Consultas ---
Total registros: 1282960
IDs únicos de clientes que consultaron: 319862
Registros sin ID de cliente: 0

Distribución de longitud de los IDs:
len_id
8     691770
9     312226
10    209736
7      51810
6      11698
11      3008
5       1290
4        760
12       140
36       138
14        98
1         94
13        84
3         64
2         40
17         2
16         2
Name: count, dtype: int64


Hallazgo de Calidad de Datos: Inconsistencia en Identificadores

Al auditar la variable ID_Cliente en el historial de consultas, detecté una dispersión anómala en la longitud de los registros:

    Datos Basura: Existen registros con identificadores de 1 a 5 dígitos (ej. 94 casos con longitud 1) y otros extremos de 17 dígitos. Esto sugiere pruebas de sistema o errores de digitación en el canal de atención.

    Distribución Real: La gran masa de datos se concentra entre los 7 y 10 dígitos, lo cual es consistente con el formato de documentos de identidad nacionales.

In [25]:
# Cruzamos el historial con los usuarios limpios (df_usuarios_clean).
df_master = pd.merge(df_consultas_raw, df_usuarios_clean, on='ID_Cuenta', how='inner')

#Limpieza inmediata de columnas basura
if 'Unnamed: 0' in df_master.columns:
    df_master.drop(columns=['Unnamed: 0'], inplace=True)

# 4Validación de Volumetría
print(f"Dimensiones del Dataset Maestro: {df_master.shape}")
print(f"Registros descartados del historial: {len(df_consultas_raw) - len(df_master)}")

Dimensiones del Dataset Maestro: (1140532, 24)
Registros descartados del historial: 142428


In [26]:
# --- Auditoría de Calidad Post-Merge ---

# A. Revisión de Tipos de Datos
print("--- Tipos de Datos ---")
print(df_master.dtypes)

# B. Detección de Valores Nulos
# Importante ver si 'Motivo_llamada' o 'Forma_pago' (que vienen de Usuarios) tienen huecos
nulls = df_master.isnull().sum()
print("\n--- Variables con Valores Nulos ---")
print(nulls[nulls > 0])

# C. Estadísticas de Variables Numéricas
# Revisamos las variables de tiempo y montos que venían en la tabla de Usuarios
cols_num = ['Monto_adeudado', 'Duracion_llamada', 'Tiempo_en_espera', 'Edad', 'Antiguedad']
print("\n--- Estadísticas Descriptivas (Numéricas) ---")
# Filtramos solo las columnas que existen en el df final para evitar errores
cols_existentes = [c for c in cols_num if c in df_master.columns]
print(df_master[cols_existentes].describe().round(2))

# D. Revisión de Categorías Clave
cols_cat = ['Tipo_consulta', 'Motivo_llamada', 'Estrato', 'y'] # Incluimos 'y' para ver balanceo
print("\n--- Distribución de Categorías Clave ---")
for col in cols_cat:
    if col in df_master.columns:
        print(f"\nVariable: {col}")
        print(df_master[col].value_counts(dropna=False).head(5))

--- Tipos de Datos ---
ID_Cuenta                 object
Fecha_consulta            object
Tipo_consulta             object
Tipo_persona              object
Departamento              object
Tiene_plan_avanzado       object
Tipo_Plan                 object
Es_moroso                 object
Tiene_plus                object
Ha_caido_mora             object
Correo                    object
usa_app                   object
Forma_pago                object
Monto_adeudado           float64
Edad                       int64
Estrato                   object
Motivo_llamada            object
Duracion_llamada         float64
Tiempo_en_espera         float64
Transferencia_llamada     object
primera_llamada           object
Antiguedad                object
Recomienda_marca          object
y                          int64
dtype: object

--- Variables con Valores Nulos ---
Tipo_persona       22952
Tipo_Plan         112924
Forma_pago        285604
Estrato           112984
Motivo_llamada      1076
dtype: in

In [ ]:
# 1. Transformación de variable temporal
# Convierto la columna a datetime. 'errors=coerce' ayuda a que si hay alguna fecha basura,
# la convierta en NaT (Not a Time) en lugar de romper el código.
df_master['Fecha_consulta'] = pd.to_datetime(df_master['Fecha_consulta'], errors='coerce')

# Verifico el rango de fechas para ver si tiene sentido
print(f"Fecha mínima: {df_master['Fecha_consulta'].min()}")
print(f"Fecha máxima: {df_master['Fecha_consulta'].max()}")

--- Nulos Restantes ---
0
--- Dimensiones Finales del Dataset Maestro ---
(1140532, 24)
